In [2]:
# import os
# os.chdir("birthrate_mtgp")
from jax import numpy as jnp
import numpy as np
import numpyro.distributions as dist
import jax.numpy as jnp
import numpyro
from numpyro.handlers import scope

## Age

In [7]:
import pandas as pd

# File paths
path_fertility = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
path_2024 = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Mother Age.csv"

# Step 1: Load both datasets
df_fert = pd.read_csv(path_fertility)
df_2024 = pd.read_csv(path_2024)

# Step 2: Standardize column names
df_2024.columns = df_2024.columns.str.strip().str.lower()
df_2024.rename(columns={'month': 'month', 'state': 'state', 'motherage': 'age', 'births': 'births'}, inplace=True)

# Step 3: Map age groups to match target columns
def map_age_group(age):
    if age in ['15-19', '20-24']:
        return 'births_age1524'
    elif age in ['25-29', '30-34']:
        return 'births_age2534'
    elif age in ['35-39', '40-44']:
        return 'births_age3544'
    else:
        return None

df_2024['age_group'] = df_2024['age'].apply(map_age_group)
df_2024 = df_2024.dropna(subset=['age_group'])

# Step 4: Compute bimonthly code (bmcode: 1 for Jan-Feb, ..., 6 for Nov-Dec)
df_2024['bmcode'] = ((df_2024['month'] - 1) // 2 + 1).astype(int)

# Step 5: Aggregate to state-bmcode-age_group level
df_agg = df_2024.groupby(['state', 'bmcode', 'age_group'])['births'].sum().unstack('age_group').reset_index()
df_agg['year'] = 2023

# Step 6: Reorder and match column order with original dataset
df_agg = df_agg[['state', 'year', 'bmcode', 'births_age1524', 'births_age2534', 'births_age3544']]

# Step 7: Replace 2023 rows in original data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_updated = pd.concat([df_fert_no2023, df_agg], ignore_index=True)

# Step 8: Save updated dataset
output_path = path_fertility.replace("fertility_data with 2024.csv", "fertility_data_updated.csv")
df_updated.to_csv(output_path, index=False)
print("✅ Updated data saved to:", output_path)


✅ Updated data saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated.csv


## Marital Status

In [3]:
import pandas as pd

# File paths
fertility_updated_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated.csv"
marital_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Marital Status.csv"

# Step 1: Load datasets
df_fert = pd.read_csv(fertility_updated_path)
df_marital = pd.read_csv(marital_path)

# Step 2: Standardize column names
df_marital.columns = df_marital.columns.str.strip().str.lower()

# Step 3: Map marital status to correct column name
df_marital['marital_col'] = df_marital['marital_status'].map({
    'Married': 'births_married',
    'Unmarried': 'births_unmarried'
})

# Drop unknown statuses
df_marital = df_marital.dropna(subset=['marital_col'])

# Step 4: Calculate bmcode (1 to 6)
df_marital['bmcode'] = ((df_marital['month'] - 1) // 2 + 1).astype(int)

# Step 5: Pivot to wide format: one row per (state, bmcode), columns: births_married, births_unmarried
df_marital_pivot = (
    df_marital
    .groupby(['state', 'bmcode', 'marital_col'])['births']
    .sum()
    .unstack('marital_col')
    .reset_index()
)

# Add year column
df_marital_pivot['year'] = 2023

# Step 6: Keep relevant columns in order
df_marital_pivot = df_marital_pivot[['state', 'year', 'bmcode', 'births_married', 'births_unmarried']]

# Step 7: Merge with previous data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]

# First merge previous age-group replacement
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['births_married', 'births_unmarried'], errors='ignore')

# Now merge with marital data
df_merged = pd.merge(
    df_fert_2023,
    df_marital_pivot,
    on=['state', 'year', 'bmcode'],
    how='left'
)

# Step 8: Combine everything
df_final = pd.concat([df_fert_no2023, df_merged], ignore_index=True)

# Step 9: Save final dataset
output_path = fertility_updated_path.replace("fertility_data_updated.csv", "fertility_data_updated_v2.csv")
df_final.to_csv(output_path, index=False)
print("✅ Final updated file saved to:", output_path)


✅ Final updated file saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v2.csv


## Insurance

In [4]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v2.csv"
insurance_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Insurance.csv"

# Step 1: Load data
df_fert = pd.read_csv(fertility_path)
df_ins = pd.read_csv(insurance_path)

# Step 2: Standardize column names
df_ins.columns = df_ins.columns.str.strip().str.lower()
if 'insurance' not in df_ins.columns:
    print("❌ Error: Could not find 'insurance' column. Found:", df_ins.columns.tolist())
    raise

# Step 3: Classify insurance types
df_ins['insurance_group'] = df_ins['insurance'].apply(
    lambda x: 'births_medicaid' if x.strip().lower() == 'medicaid' else 'births_nonmedicaid'
)

# Step 4: Compute bmcode (Jan-Feb = 1, Mar-Apr = 2, ..., Nov-Dec = 6)
df_ins['bmcode'] = ((df_ins['month'] - 1) // 2 + 1).astype(int)

# Step 5: Aggregate by state-bmcode-insurance_group
df_ins_agg = (
    df_ins
    .groupby(['state', 'bmcode', 'insurance_group'])['births']
    .sum()
    .unstack('insurance_group')
    .reset_index()
)

df_ins_agg['year'] = 2023

# Step 6: Reorder columns
df_ins_agg = df_ins_agg[['state', 'year', 'bmcode', 'births_medicaid', 'births_nonmedicaid']]

# Step 7: Separate and merge with existing 2023 data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['births_medicaid', 'births_nonmedicaid'], errors='ignore')

# Merge on keys
df_merged = pd.merge(df_fert_2023, df_ins_agg, on=['state', 'year', 'bmcode'], how='left')

# Step 8: Combine all rows and save
df_final = pd.concat([df_fert_no2023, df_merged], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v2.csv", "fertility_data_updated_v3.csv")
df_final.to_csv(output_path, index=False)
print("✅ Insurance update complete. Saved to:", output_path)


✅ Insurance update complete. Saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v3.csv


## Total

In [6]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v3.csv"
total_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Total Births.csv"

# Step 1: Load data
df_fert = pd.read_csv(fertility_path)
df_total = pd.read_csv(total_path)

# Step 2: Standardize column names
df_total.columns = df_total.columns.str.strip().str.lower()

# Step 3: Compute bimonthly code
df_total['bmcode'] = ((df_total['month'] - 1) // 2 + 1).astype(int)

# Step 4: Aggregate births by state and bmcode
df_total_agg = df_total.groupby(['state', 'bmcode'])['births'].sum().reset_index()
df_total_agg['year'] = 2023
df_total_agg.rename(columns={'births': 'births_total'}, inplace=True)

# Step 5: Prepare the 2023 portion of fertility data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['births_total'], errors='ignore')

# Step 6: Merge the new total births into 2023 portion
df_fert_2023_updated = pd.merge(df_fert_2023, df_total_agg, on=['state', 'year', 'bmcode'], how='left')

# Step 7: Combine and save
df_final = pd.concat([df_fert_no2023, df_fert_2023_updated], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v3.csv", "fertility_data_updated_v4.csv")
df_final.to_csv(output_path, index=False)
print("✅ Total births updated and saved to:", output_path)


✅ Total births updated and saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v4.csv


# Population

## Age Group

In [12]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v4.csv"
pop_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Population by Age.csv"

# Step 1: Load datasets
df_fert = pd.read_csv(fertility_path)
df_pop = pd.read_csv(pop_path)

# Step 2: Standardize column names
df_pop.columns = df_pop.columns.str.strip().str.lower()
df_pop.rename(columns={'sate': 'state', 'age group': 'age_group'}, inplace=True)

# Step 3: Map age groups to fertility categories
def map_age_group(age):
    if age in ['15-19', '20-24']:
        return 'pop_age1524'
    elif age in ['25-29', '30-34']:
        return 'pop_age2534'
    elif age in ['35-39', '40-44']:
        return 'pop_age3544'
    else:
        return None

df_pop['pop_group'] = df_pop['age_group'].apply(map_age_group)
df_pop = df_pop.dropna(subset=['pop_group'])

# Step 4: Aggregate to state + pop_group
df_pop_agg = (
    df_pop.groupby(['state', 'pop_group'])['population']
    .sum()
    .unstack('pop_group')
    .reset_index()
)
df_pop_agg['year'] = 2023  # Population for all bmcode in 2023

# Step 5: Apply this to every bmcode row in 2023 fertility data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['pop_age1524', 'pop_age2534', 'pop_age3544'], errors='ignore')

# Merge: many-to-one on state and year
df_fert_2023_updated = pd.merge(df_fert_2023, df_pop_agg, on=['state', 'year'], how='left')

# Step 6: Combine and save
df_final = pd.concat([df_fert_no2023, df_fert_2023_updated], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v4.csv", "fertility_data_updated_v5.csv")
df_final.to_csv(output_path, index=False)
print("✅ Population updated and saved to:", output_path)


✅ Population updated and saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v5.csv


## Total

In [10]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v5.csv"
pop_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Population by Age.csv"

# Step 1: Load datasets
df_fert = pd.read_csv(fertility_path)
df_pop = pd.read_csv(pop_path)

# Step 2: Standardize column names
df_pop.columns = df_pop.columns.str.strip().str.lower()
df_pop.rename(columns={'sate': 'state', 'age group': 'age_group'}, inplace=True)

# Step 3: Sum all population by state (regardless of age group)
df_total_pop = df_pop.groupby('state')['population'].sum().reset_index()
df_total_pop['year'] = 2023
df_total_pop.rename(columns={'population': 'pop_total'}, inplace=True)

# Step 4: Prepare 2023 data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['pop_total'], errors='ignore')

# Step 5: Merge total population to 2023 fertility rows
df_fert_2023_updated = pd.merge(df_fert_2023, df_total_pop, on=['state', 'year'], how='left')

# Step 6: Concatenate and save
df_final = pd.concat([df_fert_no2023, df_fert_2023_updated], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v5.csv", "fertility_data_updated_v6.csv")
df_final.to_csv(output_path, index=False)
print("✅ pop_total updated and saved to:", output_path)


✅ pop_total updated and saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v6.csv


In [ ]:
import pandas as pd

# File paths
original_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
updated_path = original_path.replace("fertility_data with 2024.csv", "fertility_data_updated_v6.csv")
final_output_path = original_path.replace("fertility_data with 2024.csv", "fertility_data_fully_updated.csv")

# Load original and updated datasets
df_original = pd.read_csv(original_path)
df_updated = pd.read_csv(updated_path)

# Separate out the 2023 data in the original file
df_original_no2023 = df_original[df_original['year'] != 2023]

# Combine with the updated 2023 data
df_updated_2023 = df_updated[df_updated['year'] == 2023]

# Concatenate and save final output
df_final = pd.concat([df_original_no2023, df_updated_2023], ignore_index=True)
df_final.to_csv(final_output_path, index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Fully Updated Fertility Data", dataframe=df_final)


In [20]:
import pandas as pd

# Re-define paths after kernel reset
original_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
updated_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v6.csv"
final_output_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_fully_updated2.csv"

# Load original and updated datasets
df_original = pd.read_csv(original_path)
df_updated = pd.read_csv(updated_path)

# Columns to update
columns_to_update = [
    'births_age1524', 'births_age2534', 'births_age3544',
    'births_married', 'births_unmarried',
    'births_medicaid', 'births_nonmedicaid',
    'births_total',
    'pop_age1524', 'pop_age2534', 'pop_age3544', 'pop_total'
]

# Separate 2023 data
df_original_2023 = df_original[df_original['year'] == 2023]
df_non_2023 = df_original[df_original['year'] != 2023]
df_updated_2023 = df_updated[df_updated['year'] == 2023][['state', 'year', 'bmcode'] + columns_to_update]

# Merge while preserving other columns
df_merged_2023 = pd.merge(df_original_2023, df_updated_2023, on=['state', 'year', 'bmcode'], how='left', suffixes=('', '_new'))

# Replace updated columns
for col in columns_to_update:
    new_col = f"{col}_new"
    if new_col in df_merged_2023.columns:
        df_merged_2023[col] = df_merged_2023[new_col]
        df_merged_2023.drop(columns=[new_col], inplace=True)

# Recombine and save
df_final = pd.concat([df_non_2023, df_merged_2023], ignore_index=True)
df_final.to_csv(final_output_path, index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Selective 2023 Update", dataframe=df_final)


ModuleNotFoundError: No module named 'ace_tools'

In [19]:
pip install ace_tools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd

# load the "total" subgroup file
df = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/fertility_results/2024/NB_births_total_6_through_june.csv")

# list all ypred columns
ypred_cols = [c for c in df.columns if c.startswith("ypred")]

# mean prediction for state 1, quarter 1
print("Mean ypred[1,1,1]:", df["ypred[1,1,48]"].mean())

# overall mean across all states & times
print("Overall mean prediction:", df[ypred_cols].values.mean())


Mean ypred[1,1,1]: 9336.086
Overall mean prediction: 12175.123730800653


In [26]:
df["ypred[1,1,48]"].mean()+df["ypred[1,1,47]"].mean()+df["ypred[1,1,46]"].mean()+df["ypred[1,1,45]"].mean()+df["ypred[1,1,44]"].mean()+df["ypred[1,1,43]"].mean()

56630.903

In [38]:
df = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/fertility_results/NB_births_age_12_through_june.csv")


In [ ]:
df = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/fertility_results/2024/NB_births_age_6_through_june.csv")


In [40]:
df["ypred[1,1,12]"].mean()+df["ypred[1,1,11]"].mean()+df["ypred[1,1,10]"].mean()+df["ypred[1,1,9]"].mean()+df["ypred[1,1,8]"].mean()+df["ypred[1,1,7]"].mean()

20020.541

In [37]:
df["ypred[2,1,12]"].mean()+df["ypred[2,1,11]"].mean()+df["ypred[2,1,10]"].mean()+df["ypred[2,1,9]"].mean()+df["ypred[2,1,8]"].mean()+df["ypred[2,1,7]"].mean()

32110.267